In [52]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import pandas as pd
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# from pandas.io.json import json_normalize

In [53]:
df = pd.read_excel("/Users/Jake/Desktop/UF/GatorsBEATWaste/data/GBW_Data_2021.xlsm",
                  sheet_name="Database")

# Calculate diversion ratio

div_rat = $\frac{weight(recycle + compost)}{weight(recycle + compost + landfill)}$

In [54]:
def calc_all_weight(df, stream):
    """Return the total weight of `stream` across all data.
    
    Parameters
    ----------
    df : DataFrame
    stream : str
        'Paper Recycling', 'Plastic Recycling', 'Compost', 'Landfill'
    """
    small_df = df[df['What bin was the item found in?'] == stream]
    return small_df.sum(axis=0)['Item Weight (pounds)']

def calc_correct_weight(df, stream):
    """Return the total weight of `stream` when labeled "Correct".
    
    Parameters
    ----------
    df : DataFrame
    stream : str
        'Paper Recycling', 'Plastic Recycling', 'Compost', 'Landfill'
    """
    bool_corr = (df['Was it found in the correct or incorrect bin?'] == "Correct")
    bool_stream = (df['What bin was the item found in?'] == stream)
    df_corr_stream = df[bool_stream & bool_corr]
    return calc_all_weight(df_corr_stream, stream)

def calc_incorrect_weight(df, stream):
    """Return the total weight of `stream` when labeled "Inorrect".
    
    Parameters
    ----------
    df : DataFrame
    stream : str
        'Paper Recycling', 'Plastic Recycling', 'Compost', 'Landfill'
    """
    all_wt = calc_all_weight(df, stream)
    cor_wt = calc_correct_weight(df, stream)
    return all_wt - cor_wt

# def calc_diverted_weight(df, stream):
#     """Return the total weight of `stream` """

def calc_div_ratio(df, mv_incorr_to_landfill=True, best_case=False):
    """Return the diversion rate (ratio) of data.
    
    Diversion ratio = sum(recycle, compost) / sum(recycle, compost, landfill)
    
    FIXME:
    [ ] Needs to put incorrect data into "Landfill" bin.
    
    NOTES:
        
    Parameters
    ----------
    df : DataFrame
    mv_incorr_to_landfill : bool
        If True, incorrect items are moved to the landfill.
        This provides a more accurate measure of the diversion
        ratio since an incorrectly recycled bottle doesn't actually
        help divert waste. It now must go in landfill.
    best_case : bool
        If True, pretend all items were correctly placed.
        Calculate this hypothetical diversion ratio. 
    """
    assert not (mv_incorr_to_landfill and best_case), "Choose one or the other!"
    
    streams = ["Paper Recycling", "Plastic Recycling", "Compost", "Landfill"]
    denom = sum([calc_all_weight(df, strm) for strm in streams])
    
    if mv_incorr_to_landfill:
        # Count incorrect items in Recycle and Compost towards landfill weight.
        numer = sum([calc_correct_weight(df, strm) for strm in streams if strm not in "Landfill"])
    elif best_case:
        # Suppose all items collected were placed in correct bins.
        raise RuntimeError("Need to put all items in correct bins.")
#         numer = sum([calc_all_weight(strm) for strm in streams if strm not in "Landfill"])
    else:
        # Plain and simple.
        numer = sum([calc_all_weight(df, strm) for strm in streams if strm not in "Landfill"])
    return numer / float(denom)

In [56]:
calc_div_ratio(df, mv_incorr_to_landfill=True, best_case=False)

0.3854554636702228

In [57]:
calc_div_ratio(df, mv_incorr_to_landfill=False, best_case=False)

0.5377185783725775

In [40]:
calc_all_weight(df, "Paper Recycling")

2.325991189427313

In [42]:
calc_incorrect_weight(df, "Paper Recycling")

1.590308370044053

In [31]:
booldf_paper = (df['What bin was the item found in?'] == "Paper Recycling")
booldf_incorr = (df['Was it found in the correct or incorrect bin?'] == "Incorrect")
booldf_corr = (df['Was it found in the correct or incorrect bin?'] == "Correct")

df_incorr_paper = df[booldf_paper & booldf_incorr]
calc_all_weight(df_incorr_paper, "Paper Recycling")


1.5903083700440528

# Item Classes

In [14]:
# PlasticItem("bottle", )

class PlasticItem:
    def __init__(self):
        self.correct_bin = "Plastic"

class PaperItem:
    def __init__(self, name):
        self.correct_bin = "Paper"
        self.name = name
        
class Restaurant:
    def __init__(self, name):
        self.name = name
        self.waste_num = 0

In [15]:
wendy = Restaurant("Wendy's")

In [20]:
wendy.waste_num = wendy.waste_num + 0.2

In [21]:
wendy.waste_num

0.7

In [ ]:
my_dct = {
    "michelle" : "A bright and young aspiring scientist who will work CDM Smith!!!",
}

In [13]:
df.iloc[1]["Unnamed: 6"]

str

In [ ]:
#Connect to service account
# scope = ['https://spreadsheets.google.com/feeds']

# credentials = ServiceAccountCredentials.from_json_keyfile_name('', scope)

# gc=gspread.authorize(credentials)

# #Get candidate data sheet from Google Drive
# spreadsheet_key = ''
# #Delete spreadsheet key before uploading

# book = gc.open_by_key(spreadsheet_key) 
# worksheet = book.worksheet("Data Analysis 2") 
# table = worksheet.get_all_values()
# print(table)

# table[0][1]

# ##Convert table data into a dataframe
# df = pd.DataFrame(table[1:], columns=table[0])
# # table[1:] means start at first element (not 0th element) and keep going
# df


# # df.iloc(0)